In [1]:
from pdf2image import convert_from_path

def pdf_to_images(pdf, filename):
    pdf = pdf.replace('\\', '\\\\')
    # print(pdf)
    images = convert_from_path(pdf)
    for i in range(len(images)):
        images[i].save('electricity_images2\\{}_page{}'.format(filename, i) +'.jpg', 'JPEG')

In [2]:
import glob
import os

for pdf in glob.glob('C:\\air_ticket\\EB READING JAN\'22 to FEB\'23\\*'):
    # print(pdf)
    # print(os.path.split(pdf))
    filename = os.path.split(pdf)[-1].split('.pdf')[0]
    filename = filename.replace(' ', '_')
    # print(filename)
    pdf_to_images(pdf, filename)

In [16]:
from PIL import Image, ImageOps

# open the image
image = Image.open("hotel_stay_images3/Netlink_Digital_Bill_No._119770,119771,119773,119774,119799,119805_page2.jpg")

# convert the image to grayscale
gray_image = image.convert('L')

# set the threshold value
threshold = 200

# convert the grayscale image to binary using the threshold value
binary_image = gray_image.point(lambda x: 0 if x < threshold else 255, '1')

# convert the binary image to mode "L"
binary_image = binary_image.convert("L")

# invert the binary image
inverted_image = ImageOps.invert(binary_image)

# create a new image with white background
new_image = Image.new("RGB", image.size, (255, 255, 255))

# composite the inverted image onto the new image with alpha channel
if len(inverted_image.split()) == 4:
    # if the image has an alpha channel, use it as the mask
    new_image.paste(image, mask=inverted_image.split()[3])
else:
    # otherwise, use the inverted image as the mask
    new_image.paste(image, mask=inverted_image)

# save the new image
new_image.save("text_image_white_background.jpg")


In [17]:
from prettyprinter import pprint

command = 'tesseract text_image_white_background.jpg file -l eng --psm 4'
os.system(command)
with open('file.txt', encoding='utf-8') as f:
    text = f.read()
text = text.splitlines()
pprint(text)

[
    'COURTYARD"',
    '',
    'BY MARRIOTT',
    '',
    'INVOICE',
    '',
    'Courtyard by Marriott, Bhopal',
    'DB City - Arera Hills Bhopal, 462011 India Bhopal 462011 P 91 755 '
        '666 1666 F 91755 3096222',
    'www.marriott.com',
    '',
    ' ',
    '',
    'DATE DESCRIPTION CHARGES CREDITS',
    '',
    ' ',
    '',
    'As a Marriott Bonvoy Member, you could have earned points toward '
        'your free dream vacation today. Start earning points and elite '
        'status,',
    'plus enjoy exclusive member offers. Enroll today at the front desk',
    '',
    'This statement is your only receipt. | agree that | am personally '
        'liable for the above-mentioned fia',
    'association indicated by me as being responsible for the payment, | '
        'understand that my liabilit',
    '',
    '     ',
    '',
    'd if the person, company or',
    'ut with such person, company',
    '',
    ' ',
    '',
    'Guest Signature Cashier Signature',
    'Checkout By

In [3]:
import cv2
import numpy
import pytesseract
from PIL import ImageGrab 

# based on: https://stackoverflow.com/questions/33949831/whats-the-way-to-remove-all-lines-and-borders-in-imagekeep-texts-programmatic

for img in glob.glob('electricity_images2\\*'):
    filename = os.path.split(img)[-1].split('.jpg')[0]
    image = cv2.imread(img)
    result = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255,
                        cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Remove horizontal lines
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
    remove_horizontal = cv2.morphologyEx(thresh,
                                        cv2.MORPH_OPEN,
                                        horizontal_kernel,
                                        iterations=2)
    cnts = cv2.findContours(remove_horizontal, cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        cv2.drawContours(result, [c], -1, (255, 255, 255), 5)

    # Remove vertical lines
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
    remove_vertical = cv2.morphologyEx(thresh,
                                    cv2.MORPH_OPEN,
                                    vertical_kernel,
                                    iterations=2)
    cnts = cv2.findContours(remove_vertical, cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        cv2.drawContours(result, [c], -1, (255, 255, 255), 5)

    preproc_image = 'remove_lines2\\'+filename+'.jpg'
    cv2.imwrite(preproc_image, result)

    output_file = 'remove_lines2\\'+filename
    command = 'tesseract {} {} -l eng --psm 4'.format(preproc_image, output_file)
    os.system(command)

    # cv2.imwrite('result.jpg', result)

In [5]:
import re
import glob
from prettyprinter import pprint
import pandas as pd
import datetime

def extract_date(text):
    start_date = ''
    end_date = ''
    for line in text:
        x = re.search('.+from.+to.+', line.lower())
        if x:
            words = line.lower().split()
            ind_from = words.index('from')
            start_date = words[ind_from+1]
            start_date = datetime.datetime.strptime(start_date, '%d/%m/%y').strftime('%Y-%m-%d')
            ind_to = words.index('to')
            end_date = words[ind_to+1]
            end_date = datetime.datetime.strptime(end_date, '%d/%m/%y').strftime('%Y-%m-%d')
    return start_date, end_date

def extract_supplier_name(text):
    supplier_name = ''
    if 'annexure' not in text[0].lower():
        supplier_name = text[0]
    return supplier_name

def extract_fuel_consumption(text):
    total_fuel = 0
    for i in range(len(text)):
        if 'particulars' in text[i].lower():
            break
    values = []
    for j in range(i+1, len(text)):
        if '--' in text[j].split()[-2] or '-' in text[j].split()[-2]:
            break
        values.append(float(text[j].split()[-2]))
        sum_values = sum(values)
    return sum_values

def extract_fuel_consumption_unit():
    return 'kWh'

start_dates = []
end_dates = []
supplier_names = []
total_fuels = []
units = []
asset_names = []
fuel_types = []

for file in glob.glob('remove_lines\\*.txt'):
    print(file)
    flag = 0
    with open(file) as fp:
        text = fp.read()
        text = text.splitlines()
        text = [line for line in text if not line.isspace() and len(line) > 0]

        for line in text:
            x = re.search('.+particulars.+units.+', line.lower())
            if x:
                flag = 1
                break
        
        if flag:
            asset_name = 'NETLINK DIGITAL SOLUTION INDIA PVT LTD'
            asset_names.append(asset_name)
            # print(asset_name)
            start_date = extract_date(text)[0]
            start_dates.append(start_date)
            # print(start_date)
            end_date = extract_date(text)[1]
            end_dates.append(end_date)
            # print(end_date)
            supplier_name = extract_supplier_name(text)
            supplier_names.append(supplier_name)
            # print(supplier_name)
            total_fuel = extract_fuel_consumption(text)
            total_fuels.append(total_fuel)
            # print(total_fuel)
            unit = extract_fuel_consumption_unit()
            units.append(unit)
            # print(unit)
            fuel_type = 'Electricity'
            fuel_types.append(fuel_type)
            # print(fuel_type)
    # print('----------------------------------------')

df = pd.DataFrame({
    'Asset Name': asset_names,
    'Start Date': start_dates,
    'End Date': end_dates,
    'Fuel Consumption': total_fuels,
    'Fuel Consumption Unit': units,
    'Fuel Type': fuel_types,
    'Supplier': supplier_names
})

df.to_excel('netlink_electricity_data.xlsx')

remove_lines\Chennai_Office_EB_Apr_22_page0.txt
remove_lines\Chennai_Office_EB_Apr_22_page1.txt
remove_lines\Chennai_Office_EB_Apr_22_page2.txt
remove_lines\Chennai_Office_EB_Apr_22_page3.txt
remove_lines\Chennai_Office_EB_Apr_22_page4.txt
remove_lines\Chennai_Office_EB_Aug_22_1_page0.txt
remove_lines\Chennai_Office_EB_Aug_22_1_page1.txt
remove_lines\Chennai_Office_EB_Aug_22_1_page2.txt
remove_lines\Chennai_Office_EB_Aug_22_page0.txt
remove_lines\Chennai_Office_EB_Aug_22_page1.txt
remove_lines\Chennai_Office_EB_Aug_22_page2.txt
remove_lines\Chennai_Office_EB_Aug_22_page3.txt
remove_lines\Chennai_Office_EB_Dec_22_1_page0.txt
remove_lines\Chennai_Office_EB_Dec_22_1_page1.txt
remove_lines\Chennai_Office_EB_Dec_22_1_page2.txt
remove_lines\Chennai_Office_EB_Dec_22_1_page3.txt
remove_lines\Chennai_Office_EB_Dec_22_page0.txt
remove_lines\Chennai_Office_EB_Dec_22_page1.txt
remove_lines\Chennai_Office_EB_Dec_22_page2.txt
remove_lines\Chennai_Office_EB_Dec_22_page3.txt
remove_lines\Chennai_Offic

In [ ]:
image = 'electricity_images\\Chennai_Office_EB_Apr_22_page0.jpg'
command = 'tesseract {} stdout -l eng --psm 4'.format(image)
print(os.popen(command).read())

In [43]:
from img2table.ocr import TesseractOCR
from img2table.document import Image
from prettyprinter import pprint
import cv2
from PIL import Image as Img

image = Image('preprocessed_electricity\\Chennai_Office_EB_Apr_22_page0_set_dpi-convert_to_rgb.jpg')
ocr = TesseractOCR(lang="eng")
image_tables = image.extract_tables(ocr=ocr)
pprint(image_tables)

extracted_tables = image.extract_tables(ocr=ocr,
                                      implicit_rows=True,
                                      borderless_tables=False,
                                      min_confidence=50)
# pprint(extracted_tables)
# print(image.content)
# image.to_xlsx('tables_image1.xlsx',
#             ocr=ocr)


# img = cv2.imread('preprocessed_electricity\\Chennai_Office_EB_Apr_22_page0_set_dpi-convert_to_rgb.jpg')
# img = img[822:1369, 181:1473]
# cv2.imwrite('img.jpg', img)
# # cv2.imshow('img', img)
# # cv2.waitKey()
# im = Img.open('img.jpg')
# im.save('img.jpg', dpi=(1200,1200))
# img = cv2.imread('img.jpg')
# img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# cv2.imwrite('img.jpg', img)

# command = 'tesseract img.jpg stdout -l eng --psm 4'
# print(os.popen(command).read())
# # # Instantiation of the pdf
# # pdf = PDF(src="Leased Asset Emission Management\\2022\\Chennai Office EB\\Chennai Office EB Apr_22.pdf")

# # # Instantiation of the OCR, Tesseract, which requires prior installation
# # ocr = TesseractOCR(lang="eng")

# # # Table identification and extraction
# # pdf_tables = pdf.extract_tables(ocr=ocr)

# # # We can also create an excel file with the tables
# # pdf.to_xlsx('tables.xlsx',
# #             ocr=ocr)

[ExtractedTable(title=None, bbox=(201, 842, 1453, 1349),shape=(8, 4))]


In [ ]:
!pip install img2table

In [10]:
from img2table.document import Image

# Instantiation of the image
img = Image(src="electricity_images\\Chennai_Office_EB_Apr_22_page0.jpg")

# Table identification
img_tables = img.extract_tables()

# Result of table identification
print(img_tables)

[ExtractedTable(title=None, bbox=(201, 842, 1453, 1349),shape=(12, 4))]


In [18]:
import cv2
import pytesseract

# Load image and convert to grayscale
img = cv2.imread('electricity_images\\Chennai_Office_EB_Apr_22_page0.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply thresholding to enhance table edges
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Apply erosion and dilation to remove noise
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

# Find contours of table
contours, hierarchy = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Determine contour that represents the table
table_contour = None
for contour in contours:
    x,y,w,h = cv2.boundingRect(contour)
    if w > 50 and h > 50:
        table_contour = contour
        break

# Extract table image
if table_contour is not None:
    x,y,w,h = cv2.boundingRect(table_contour)
    table_img = img[y:y+h, x:x+w]

    # Apply OCR to table image
    text = pytesseract.image_to_string(table_img)

    # Print extracted text
    print(text)
else:
    print('No table found in image')


In [46]:
import cv2
import numpy
import pytesseract
from PIL import ImageGrab 

# based on: https://stackoverflow.com/questions/33949831/whats-the-way-to-remove-all-lines-and-borders-in-imagekeep-texts-programmatic

image = 'electricity_images\\Chennai_Office_EB_Apr_22_page0.jpg'
image = cv2.imread(image)
result = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255,
                       cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Remove horizontal lines
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
remove_horizontal = cv2.morphologyEx(thresh,
                                     cv2.MORPH_OPEN,
                                     horizontal_kernel,
                                     iterations=2)
cnts = cv2.findContours(remove_horizontal, cv2.RETR_EXTERNAL,
                        cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    cv2.drawContours(result, [c], -1, (255, 255, 255), 5)

# Remove vertical lines
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
remove_vertical = cv2.morphologyEx(thresh,
                                   cv2.MORPH_OPEN,
                                   vertical_kernel,
                                   iterations=2)
cnts = cv2.findContours(remove_vertical, cv2.RETR_EXTERNAL,
                        cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    cv2.drawContours(result, [c], -1, (255, 255, 255), 5)

cv2.imwrite('result.jpg', result)

True

In [47]:
command = 'tesseract {} stdout -l eng --psm 4'.format('result.jpg')
print(os.popen(command).read())

RITHWIK FACILITY MANAGEMENT SERVICES LTD

Regd Office:RR Tower III, Thiru-Vi-Ka,Ind1.Estate,Guindy,Chennai-600032.
Phone: 91 - 44 - 4353 4441, Email : info@rithwik.co.in

GST NO :33AAECR6158H1ZB, CIN :U74900TN2010PLC074294

NETLINK DIGITAL SOLUTION INDIA PVT LTD,
GROUND FLOOR,RISHABH INFO PARK (P) LTD,
RR TOWER IV,TVK Industrial Estate,Guindy
CHENNAI,
TamilNadu,CODE:33
GST NO:33AADCE6269P2Z3
Invoice No:NETLINK/01/2022-23
Bill Date:03/05/2022
Due Date:09/05/2022

Summary Sheet From 01/04/22 00:00 To 30/04/22 23:59

Sno Particulars Units Amount
1 Electrical Unit Charges @ RS 8.00 11861.85 94894.80
2 DG Unit Charges @ RS 30.04 76.25 2290.55
3 Common Area Charges 0.00 5580.13
4 Peak Hour Charges -- 0.00
5 Maximum Demand Charges @ RS 350.00 65.00 22750.00
6 Meter Rent - 500.00
7 Electricity Tax -- 5882.24
8 Water Charges -- 4578.60
9 Other Charges(+/-) -- 0.00
10 Penalty Charges -- 0.00

Total Rs 136476

Amount in Words
One Lakh Thirty Six Thousand Four Hundred and Seventy Six Only

 

Auth